# LlamaIndex integration

In this notebook, we show how it's easy to build **ChainML** agents that leverage the power of **LlamaIndex** to integrate data for your agents.

## Setup

Integration with **LlamaIndex** is easy and straightforward.
To use **LlamaIndex** with the **ChainML** framework, you will need to install "Llama-Index" via pip.

### Example

```sh
$ pip install Llama-Index
```

In [1]:
# Load environment variables
import dotenv

dotenv.load_dotenv()

True

Example of using Llama Index to retrieve relevant information from documents - here we use the book, the Great Gatbsy

In [2]:
# download Great Gatsby example from Llama Index

import os
import requests

url = "https://github.com/jerryjliu/llama_index/blob/main/examples/gatsby/gatsby_full.txt"
filename = url.split("/")[-1]

os.makedirs("gatsby_download", exist_ok=True)

response = requests.get(url)
with open(os.path.join("gatsby_download", filename), "wb") as f:
    f.write(response.content)

In [3]:
from llama_index import VectorStoreIndex, SimpleDirectoryReader

# build index of book
documents = SimpleDirectoryReader("gatsby_download").load_data()
index = VectorStoreIndex.from_documents(documents)

In [17]:
query_engine = index.as_query_engine()

In [16]:
# check the index is working
response = query_engine.query("Where do Gatsby and Daisy meet?")
print(response)


Gatsby and Daisy meet at Gatsby's mansion, which is located in East Egg.


## Examples
### Chain Using LlamaIndex

Let's create a chain that uses LlamaIndex to retrieve relevant context for a user's query for a simple chatbot that can query the data. We will look up context then feed it into a prompt template for an LLMSkill to respond.

In [29]:
from chainml_engine.core import SkillBase, Budget
from chainml_engine.core.execution_context import SkillContext, SkillMessage

from llama_index.indices.query.base import BaseQueryEngine


class LlamaIndexSkill(SkillBase):
    queryEngine: BaseQueryEngine

    def __init__(self, queryEngine: BaseQueryEngine):
        SkillBase.__init__(self, "llama index skill")
        self.queryEngine = queryEngine

    def execute(self, context: SkillContext, budget: Budget) -> SkillMessage:
        prompt = context.chatHistory.last_user_message().unwrap("no user message").message
        print(prompt)
        response = self.queryEngine.query(prompt)
        return self.build_success_message(response)

In [30]:
# wrap into a trivial agent that just answers document queries
from chainml_engine.skill import LLMSkill
from chainml_engine.core import Chain, Agent
from chainml_engine.controller.basic_controller import BasicController
from chainml_engine.evaluator.basic_evaluator import BasicEvaluator

index_skill = LlamaIndexSkill(query_engine)
chain = Chain("docindex", "document index", [index_skill])
agent = Agent(BasicController(), [chain], BasicEvaluator())

In [32]:
from chainml_engine.llm.llm_message import LLMMessage
from chainml_engine.core.execution_context import AgentContext, ChatHistory

chat_history = ChatHistory()
# chat_history.add_user_message(message="Whose eyes are on the billboard?")
# chat_history.add_user_message("What are the personalities of Tom and Daisy?")
# chat_history.add_user_message("What era does the book take place in?")
chat_history.add_user_message("Who falls in love with Daisy?")
context = AgentContext(chat_history=chat_history)
result = agent.execute(context=context, budget=Budget(20))
print(result.messages[-1].message.message)

Who falls in love with Daisy?

Gatsby falls in love with Daisy.


In [49]:
# agent to use index to provide context for more complex answers
from chainml_engine.llm import OpenAILLM, OpenAIConfiguration
import dotenv
from chainml_engine.prompt import PromptBuilder
from chainml_engine.prompt.prompt_builder import PromptToMessages

dotenv.load_dotenv()
config = OpenAIConfiguration.from_env()
config.model = "gpt-3.5-turbo"
llm = OpenAILLM(config)

context_prompt = PromptToMessages(
    PromptBuilder("Please identify query terms to respond to the following user request {{chat_history.last_message}}")
)
context_query_skill = LLMSkill(
    llm,
    "You are an expert in the Great Gatbsy. Identify relevant query terms to search for context in the book.",
    context_messages=context_prompt.get_messages_from_prompt,
)

index_skill = LlamaIndexSkill(query_engine)
index_prompt = PromptToMessages(
    PromptBuilder(
        "Here are relevant quotes from the book: {{chain_history.last_message}} \nUse this to respond to the following user request {{chat_history.last_message}}"
    )
)
response_skill = LLMSkill(
    llm,
    "You are an expert in the Great Gatbsy. Provide a helpful response to the user's question",
    context_messages=index_prompt.get_messages_from_prompt,
)

chain = Chain("docindex", "document index", [context_query_skill, index_skill, response_skill])
agent = Agent(BasicController(), [chain], BasicEvaluator())

In [53]:
import logging

logging.basicConfig(
    format="[%(asctime)s %(levelname)s %(threadName)s %(name)s:%(funcName)s:%(lineno)s] %(message)s",
    datefmt="%Y-%m-%d %H:%M:%S%z",
)
## uncomment me to see the engine logs
logging.getLogger("chainml_engine").setLevel(logging.WARN)

In [54]:
chat_history = ChatHistory()
# chat_history.add_user_message(message="Whose eyes are on the billboard?")
# chat_history.add_user_message("What are the personalities of Tom and Daisy?")
# chat_history.add_user_message("What era does the book take place in?")
chat_history.add_user_message("What are the key plot events in the book?")
# chat_history.add_user_message("What is the significance of the eyes on the billboard?")
context = AgentContext(chat_history=chat_history)
result = agent.execute(context=context, budget=Budget(20))
print(result.messages[-1].message.message)

What are the key plot events in the book?
In "The Great Gatsby," there are several key plot events that drive the narrative:

1. The heated argument and confrontation between the narrator and Tom Buchanan at the party in New York City.
2. Myrtle Wilson recounting the story of her first meeting with Tom, hinting at their affair.
3. Tom Buchanan breaking Myrtle's nose during a passionate argument.
4. The small dog sitting on the table during the party, representing the reckless and careless behavior of the characters.
5. The narrator wiping the spot of dried lather from Mr. McKee's cheek, symbolizing his tendency to get involved in others' affairs.
6. A heated argument between Tom Buchanan and Myrtle Wilson over Daisy's name, revealing tensions and resentment.
7. Jordan Baker emerging from the house and standing at the head of the marble steps, capturing the attention of the narrator.
8. The narrator and Jordan Baker connecting and exploring the garden together.
9. Conversations between 